In [9]:
# ODL Live API - Connection and Optimization
# Simple notebook for connecting to ODL and getting optimized routes

## Setup
import requests
import json
from datetime import datetime


In [34]:
# INSERT CREDENTIALS HERE


In [14]:
## Initialize session
session = requests.Session()
session.auth = (USERNAME, PASSWORD)
session.headers.update({
    'Content-Type': 'application/json',
    'Accept': 'application/json'
})

print("✓ Session configured")
print(f"Base URL: {BASE_URL}")

✓ Session configured
Base URL: https://optimizer-0.staging.zenderatms.com


In [13]:
## Load your model
# Adjust path to where your test_model_mathias.json is located
MODEL_PATH = "../test_model_mathias_fixed.json"

with open(MODEL_PATH, 'r') as f:
    model = json.load(f)

print(f"\n✓ Model loaded")
print(f"Jobs: {len(model['data']['jobs'])}")
print(f"Vehicles: {len(model['data']['vehicles'])}")




✓ Model loaded
Jobs: 106
Vehicles: 26


In [35]:
## PUT model to ODL (upload and start optimization)
MODEL_ID = f"test_{datetime.now().strftime('%Y%m%d_%H%M%S')}"  # Unique ID

url = f"{BASE_URL}/models/{MODEL_ID}"

print(f"\nUploading model to: {url}")
response = session.put(url, json=model, timeout=60)

print(f"Status: {response.status_code}")

if response.status_code in [200, 201, 204]:
    print("✓ Model uploaded successfully!")
    print("✓ Optimization is running in background")
else:
    print(f"✗ Error: {response.text}")




Uploading model to: https://optimizer-0.staging.zenderatms.com/models/test_20251028_120353
Status: 200
✓ Model uploaded successfully!
✓ Optimization is running in background


In [36]:
## GET optimized plan (wait a bit for optimization)
import time

print("\nWaiting 10 seconds for initial optimization...")
time.sleep(10)

plan_url = f"{BASE_URL}/models/{MODEL_ID}/optimiserstate/plan"
plan_response = session.get(plan_url, timeout=30)

if plan_response.status_code == 200:
    plan = plan_response.json()
    print("✓ Plan retrieved!")
    
    # Save to file
    output_file = f'plan_{MODEL_ID}.json'
    with open(output_file, 'w') as f:
        json.dump(plan, f, indent=2)
    print(f"✓ Saved to: {output_file}")
else:
    print(f"✗ Error getting plan: {plan_response.status_code}")
    plan = None



Waiting 10 seconds for initial optimization...
✓ Plan retrieved!
✓ Saved to: plan_test_20251028_120353.json


In [18]:
## Quick analysis
if plan:
    vehicle_plans = plan.get('vehiclePlans', [])
    print(f"\n=== RESULTS ===")
    print(f"Number of vehicles: {len(vehicle_plans)}")
    
    # Count vehicles with routes
    vehicles_with_routes = [v for v in vehicle_plans if len(v.get('plannedStops', [])) > 0]
    print(f"Vehicles with routes: {len(vehicles_with_routes)}")
    
    # Total stops
    total_stops = sum(len(v.get('plannedStops', [])) for v in vehicle_plans)
    print(f"Total stops: {total_stops}")
    
    # Calculate total distance and time from timeStatistics
    total_distance_m = 0
    total_travel_seconds = 0
    total_cost = 0
    
    for vehicle in vehicle_plans:
        time_stats = vehicle.get('timeStatistics', {})
        total_distance_m += time_stats.get('travelMetres', 0)
        total_travel_seconds += time_stats.get('travelSeconds', 0)
        total_cost += time_stats.get('cost', 0)
    
    total_distance_km = total_distance_m / 1000
    total_travel_hours = total_travel_seconds / 3600
    
    print(f"Total distance: {total_distance_km:.2f} km")
    print(f"Total travel time: {total_travel_hours:.2f} hours")
    print(f"Total cost: {total_cost:.2f}")
    
    # Check unassigned jobs
    unplanned = plan.get('unplannedJobs', [])
    if len(unplanned) > 0:
        print(f"⚠ Unplanned jobs: {len(unplanned)}")
    else:
        print(f"✓ All jobs planned successfully!")


=== RESULTS ===
Number of vehicles: 26
Vehicles with routes: 8
Total stops: 172
Total distance: 999.89 km
Total travel time: 30.42 hours
Total cost: 473381.04
⚠ Unplanned jobs: 20


In [ ]:
## Clean up (delete model from server)
# Uncomment to delete:
delete_response = session.delete(url)
print(f"\nModel deleted: {delete_response.status_code}")